In [1]:
import time
import pygame
import pygame.midi
import numpy

pygame 2.1.2 (SDL 2.0.18, Python 3.9.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
def print_midi_info():
    pygame.midi.init()
    for i in range(pygame.midi.get_count()):
        print(pygame.midi.get_device_info(i))
    
    print("Default in: ", pygame.midi.get_default_input_id())
    print("Default out: ", pygame.midi.get_default_output_id())
    pygame.midi.quit()

print_midi_info()

(b'MMSystem', b'Microsoft MIDI Mapper', 0, 1, 0)
(b'MMSystem', b'AKM322', 1, 0, 0)
(b'MMSystem', b'CoolSoft MIDIMapper', 0, 1, 0)
(b'MMSystem', b'Microsoft GS Wavetable Synth', 0, 1, 0)
(b'MMSystem', b'VirtualMIDISynth #1', 0, 1, 0)
(b'MMSystem', b'AKM322', 0, 1, 0)
Default in:  1
Default out:  0


In [3]:
def take_input(in_id = None, out_id = None):
    pygame.midi.init()

    if not in_id:
        in_id = pygame.midi.get_default_input_id()

    if not out_id:
        out_id = pygame.midi.get_default_output_id()

    i = pygame.midi.Input(in_id)
    o = pygame.midi.Output(out_id,latency=0,buffer_size=1)
    o.set_instrument(0)

    pygame.display.set_mode((1,1))
    note_count = 0

    #Turn these arrays into nupy.array()s like this: final_list = np.array(list(zip(note_seq,dur_seq)))
                # Will need to map ints from keyboard to chars representing the note (ie 'C3' 'A-4' etc) 
                # then return it and in produce_note_model create a new function and map the chars to their 
                # respective ints before sending them to produce_song 

                # have to caulculate the differnece in the start and end of notes, round the result to the 
                # nearest musical "pause" then send the word along with the string for note (bpm 120)
            #Chords?
                # PLeASe nO
        #I'll need to store the notes that have been pressed to check timestamp for release of the note


        #A mid C is 60, so using mod 12 to get notes and octave is important.
    note_seq = [] 
    dur_seq = []

    while note_count < 10:                
        if i.poll(): #if a note has been presseed
            midi_event = i.read(1) #grab note played

            if midi_event[0][0][0] == 144: #if note on
                print(midi_event)
                note = midi_event[0][0][1]
                time_stamp = midi_event[0][1] # in milliseconds I think
                o.note_on(note, 120)
               

            if midi_event[0][0][0] == 128: #if note off
                note = midi_event[0][0][1]
                time_stamp = midi_event[0][1]
                o.note_off(note, 120)
               
                note_count += 1  #DOES NOT ACCOUNT FOR CHORDS

            #o.write_short(midi_event[0][0][0], midi_event[0][0][1], 120) #write start/stop, note, velocity to output

    time.sleep(1) #Give last note time to play
    i.close()
    o.close()
    pygame.midi.quit()
    pygame.quit()

take_input(out_id=4)

[[[144, 59, 62, 0], 86687]]
[[[144, 52, 12, 0], 87171]]
[[[144, 59, 88, 0], 87537]]
[[[144, 52, 69, 0], 87800]]
[[[144, 60, 96, 0], 87983]]
[[[144, 52, 85, 0], 88198]]
[[[144, 59, 102, 0], 88359]]
[[[144, 60, 90, 0], 88974]]
[[[144, 59, 73, 0], 89542]]
[[[144, 52, 63, 0], 89931]]
[[[144, 60, 102, 0], 90084]]
[[[144, 59, 127, 0], 90377]]
[[[144, 60, 127, 0], 90641]]
[[[144, 53, 90, 0], 90773]]
[[[144, 59, 127, 0], 90923]]
[[[144, 53, 102, 0], 91054]]
[[[144, 60, 127, 0], 91189]]


KeyboardInterrupt: 